In [ ]:
# ! pip install kagglehub
# ! pip install matplotlib

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

In [ ]:
df = pd.read_csv(path + "/fashion-mnist_train.csv")

df.head()

In [ ]:
sample_df = df.sample(n=30000, random_state=42)

In [ ]:
sample_df.shape

In [ ]:
X = sample_df.iloc[:, 1:].values
y = sample_df.iloc[:, :1].values


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
st_scaler = StandardScaler()

In [ ]:

X_train = st_scaler.fit_transform(X_train)
X_test = st_scaler.transform(X_test)

In [ ]:
X_train

# Dataset Class

In [ ]:
# create custom dataset class

class FashionMNISTDataset(Dataset):

    def __init__(self, features, labels):
        print(type(features))
        print(type(labels))
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)


    def __len__(self):
        return len(self.features)


    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [ ]:
train_dataset = FashionMNISTDataset(X_train, y_train)
test_dataset = FashionMNISTDataset(X_test, y_test)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# OPTUNA

In [ ]:
class MyNN(nn.Module):

    def __init__(self, input_dim, output_dim, hidden_layers, neurons_per_layer):

        super().__init__()

        layers = []

        for i in range(hidden_layers):

            layers.append(
                nn.Linear(input_dim , neurons_per_layer)
            )
            input_dim = neurons_per_layer

        layers.append(
            nn.Linear(neurons_per_layer, output_dim)
        )



        self.model = nn.Sequential(

            nn.Linear(num_features, 128),
            nn.BatchNorm1d(128), # batch normalization layer to stabilize and accelerate training, applied before activation functions, 
                                 #       128 - is the number of features from the previous layer, 1d as we have 1 dimensional data
            nn.ReLU(),
            nn.Dropout(p=0.2), # using dropout to prevent overfitting, dropout rate of 20%, used to prevent overfitting, applied after activation functions
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(64, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(16, 10),
            # nn.Softmax(dim=1) - softmax is not required in PyTorch as it is included in the CrossEntropyLoss
        )

    def forward(self, x):
        return self.model(x)



In [ ]:
# objective function

def objective_function(trial):

    # next hyperparameter values
    num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 5, step=1, log=False)
    neurons_per_layer = trial.suggest_int("neurons_per_layer", 10, 128, step=8, log=False)


    # model init
    input_dim = 784
    output_dim = 10

    class 



    # params init

    # training loop

    # evaluation


    # return accuracy
    return accuracy